# Sistema de Preguntas y Respuestas sobre Legislación Paraguaya

## Alumnos

1.   Oscar Ortíz
2.   Guillermo Osorio

In [1]:
# -*- coding: utf-8 -*-
# Script para realizar consultas a una base de datos legal paraguaya usando recuperación aumentada por generación (RAG)

import os
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from dotenv import load_dotenv

# Cargar variables de entorno desde .env (incluye clave API de OpenAI)
load_dotenv()

# Cargar el índice FAISS previamente generado con embeddings de HuggingFace
vectorstore = FAISS.load_local(
    "vectorstore_faiss",
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# Definir el modelo de lenguaje de OpenAI (GPT-3.5 Turbo en este caso)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Plantilla personalizada para generar respuestas jurídicas basadas en el contexto legal
plantilla = """Eres un asistente jurídico entrenado en legislación paraguaya. Usá exclusivamente la información de los artículos legales proporcionados para responder.

Pregunta: {question}

Artículos relevantes:
{context}

Respuesta:
"""

# Construcción del prompt
prompt = PromptTemplate(
    template=plantilla,
    input_variables=["question", "context"]
)

# Encadenar el modelo LLM con el prompt definido
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Crear una cadena para combinar documentos usando el LLM
qa_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

# Envolverlo todo en una cadena RetrievalQA
retrieval_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=qa_chain,
    return_source_documents=True
)

# Función para buscar y responder una pregunta
def buscar_y_responder(pregunta):
    resultado = retrieval_chain.invoke({"query": pregunta})
    respuesta = resultado["result"]
    fuentes = resultado.get("source_documents", [])

    if not respuesta.strip():
        respuesta = "No se encontró información relevante en la legislación disponible."

    print("\n📝 Respuesta:\n", respuesta)
    print("\n🔎 Fuente(s):")
    for doc in fuentes:
        meta = doc.metadata
        print(f" - Ley {meta.get('nro_ley')} - {meta.get('titulo_ley')} - Artículo {meta.get('nro_articulo')}")

# Demostración de uso
print("📚 Sistema de Consulta sobre Legislación Paraguaya")
print("Escribí tu pregunta o 'salir' para terminar.")

# Ejecución interactiva (simulado aquí con preguntas de ejemplo del archivo output.txt)
preguntas = [
    "¿Qué leyes me hablan sobre forma de gobierno?",
    "¿Qué leyes me hablan sobre niñez o infancia?",
    "¿Qué leyes me hablan sobre educación?",
    "¿Qué leyes me hablan sobre salud?",
    "¿Qué leyes me hablan sobre la vida en las estrellas?",
    "¿Qué disposiciones se encuentran en la de defensa del consumidor cuando se vende algo defectuoso?",
    "¿El aborto está permitido?",
    "¿Está permitido protestar en la vía pública?",
    "¿Puedo negarme a prestar servicio militar?",
    "¿Tengo derecho a atención médica gratuita?",
    "salir"
]

for pregunta in preguntas:
    if pregunta.lower() in ["salir", "exit"]:
        break
    print(f"➤ Pregunta: >? {pregunta}")
    buscar_y_responder(pregunta)

/tmp/ipykernel_345232/1462965967.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
/home/gosorio/Projects/legislacionParaguaya/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_345232/1462965967.py:53: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  

📚 Sistema de Consulta sobre Legislación Paraguaya
Escribí tu pregunta o 'salir' para terminar.
➤ Pregunta: >? ¿Qué leyes me hablan sobre forma de gobierno?

📝 Respuesta:
 Las leyes que hablan sobre la forma de gobierno en Paraguay son la Ley 1 - Constitución Nacional - Artículo 1, que establece que la República del Paraguay adopta para su gobierno la democracia representativa, participativa y pluralista, y la Ley 1 - Constitución Nacional - Artículo 161 y 167, que establecen la forma de gobierno municipal y departamental en el país.

🔎 Fuente(s):
 - Ley 1 - Constitución Nacional - Artículo Artículo 12
 - Ley 1 - Constitución Nacional - Artículo Artículo 167
 - Ley 1 - Constitución Nacional - Artículo Artículo 161
 - Ley 1 - Constitución Nacional - Artículo Artículo 1
➤ Pregunta: >? ¿Qué leyes me hablan sobre niñez o infancia?

📝 Respuesta:
 Las leyes que hablan sobre niñez o infancia en Paraguay son la Ley 1680 - Código de la Niñez y la Adolescencia. En dicha ley, se establece que la t